# Creating and Testing Functions as Tools from Notebook

## Financial Datasets API

MCP-server Repo with more functions available [here](https://github.com/financial-datasets/mcp-server/tree/main)

In [0]:
%pip install httpx httpx mcp dotenv databricks-sdk
dbutils.library.restartPython()

In [0]:
import json
import os
import httpx
import logging
import sys
from mcp.server.fastmcp import FastMCP
from dotenv import load_dotenv

### Parameters

In [0]:
os.chdir('/Workspace/Users/gabriele.albini@databricks.com/mcp-demo')
print(os.getcwd())

In [0]:
load_dotenv()
my_API_key = os.getenv('my_API_key')
FINANCIAL_DATASETS_API_BASE = "https://api.financialdatasets.ai"

### Defining Tools as Functions

In [0]:
# Helper function to make API requests
async def make_request(url: str) -> dict[str, any] | None:
    """Make a request to the Financial Datasets API with proper error handling."""
    # Load environment variables from .env file
    #load_dotenv()
    
    headers = {}
    """
    if api_key := os.environ.get("FINANCIAL_DATASETS_API_KEY"):
        headers["X-API-KEY"] = api_key
    """
    api_key = my_API_key

    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=45.0)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            return {"Error": str(e)}

In [0]:
# Tool to retrieve live stock prices
#@mcp.tool()
async def get_current_stock_price(ticker: str) -> str:
    """Get the current / latest price of a company.

    Args:
        ticker: Ticker symbol of the company (e.g. AAPL, GOOGL)
    """
    # Fetch data from the API
    url = f"{FINANCIAL_DATASETS_API_BASE}/prices/snapshot/?ticker={ticker}"
    data = await make_request(url)

    # Check if data is found
    if not data:
        return "Unable to fetch current price or no current price found."

    # Extract the current price
    snapshot = data.get("snapshot", {})

    # Check if current price is found
    if not snapshot:
        return "Unable to fetch current price or no current price found."

    # Stringify the current price
    return json.dumps(snapshot, indent=2)

In [0]:
# Tool to retreive stock prices for a defined period
#@mcp.tool()
async def get_historical_stock_prices(
    ticker: str,
    start_date: str,
    end_date: str,
    interval: str = "day",
    interval_multiplier: int = 1,
) -> str:
    """Gets historical stock prices for a company.

    Args:
        ticker: Ticker symbol of the company (e.g. AAPL, GOOGL)
        start_date: Start date of the price data (e.g. 2020-01-01)
        end_date: End date of the price data (e.g. 2020-12-31)
        interval: Interval of the price data (e.g. minute, hour, day, week, month)
        interval_multiplier: Multiplier of the interval (e.g. 1, 2, 3)
    """
    # Fetch data from the API
    url = f"{FINANCIAL_DATASETS_API_BASE}/prices/?ticker={ticker}&interval={interval}&interval_multiplier={interval_multiplier}&start_date={start_date}&end_date={end_date}"
    data = await make_request(url)

    # Check if data is found
    if not data:
        return "Unable to fetch prices or no prices found."

    # Extract the prices
    prices = data.get("prices", [])

    # Check if prices are found
    if not prices:
        return "Unable to fetch prices or no prices found."

    # Stringify the prices
    return json.dumps(prices, indent=2)

### Testing functions from notebook

In [0]:
result = await get_current_stock_price("NVDA")
print(result)

In [0]:
result = await get_historical_stock_prices("AAPL", start_date = "2025-06-25", end_date = "2025-06-30")
print(result)

In [0]:
result = await get_historical_stock_prices("GOOGL", start_date = "2025-06-25", end_date = "2025-06-25")
print(result)

## Unity Catalog Tools
Creating function as tools on UC, to leverage managed MCP server.

### Creating a new catalog and schema

In [0]:
catalog_ = "gabriele_new_catalog_tests"
schema_ = "portfolio"
my_user_ = "`gabriele.albini@databricks.com`"

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS " + catalog_)
spark.sql("USE CATALOG " + catalog_)
spark.sql("CREATE SCHEMA IF NOT EXISTS " + schema_)
spark.sql("USE SCHEMA " + schema_)

### Creating UC functions

In [0]:
def evaluate_trend(start_open: float, end_close: float) -> float:
    """
    Calculates the percentage change in an asset's value over a period.

    Args:
        start_open (float): The asset's opening value at the start of the period.
        end_close (float): The asset's closing value at the end of the period.

    Returns:
        float: The percentage change in the asset's value during the period.
               Positive values indicate a gain; negative values indicate a loss.

    Raises:
        ZeroDivisionError: If start_open is zero.
    """
    if start_open == 0:
        raise ZeroDivisionError("start_open cannot be zero.")
    trend = 100*((end_close - start_open) / start_open)
    return trend

In [0]:
print(evaluate_trend(6.93, 177.87))

In [0]:
## Register function on UC
function_comment = "This function calculates the percentage change in an asset value over a period. Args: (1) start_open (float): The asset opening value at the start of the period; (2) end_close (float): The asset closing value at the end of the period.    Returns: float: The percentage change in the asset value during the period. Positive values indicate a gain; negative values indicate a loss. Raises: ZeroDivisionError: If start_open is zero."

sql_def = f"""
CREATE OR REPLACE FUNCTION {catalog_}.{schema_}.evaluate_trend(
  start_open DOUBLE,
  end_close DOUBLE
)
RETURNS DOUBLE
LANGUAGE PYTHON
COMMENT '{function_comment}'
AS $$
if start_open == 0:
    raise ZeroDivisionError("start_open cannot be zero.")
trend = 100*((end_close - start_open) / start_open)
return trend
$$;
"""

spark.sql(sql_def)

In [0]:
spark.sql("GRANT USE CATALOG ON CATALOG " + catalog_ + " TO " + my_user_)
spark.sql("GRANT USE SCHEMA ON SCHEMA " + schema_ + " TO " + my_user_)
spark.sql("GRANT EXECUTE ON FUNCTION " + catalog_+"."+schema_+".evaluate_trend TO "+ my_user_)

In [0]:
%sql
select evaluate_trend(6.93, 177.87);

In [0]:
# Transfer ownership rights of the function and remove execute rights
#spark.sql("ALTER FUNCTION " + catalog_+"."+schema_+".evaluate_trend OWNER TO `christophe.chieu@databricks.com`")
#spark.sql("REVOKE EXECUTE ON FUNCTION " + catalog_+"."+schema_+".evaluate_trend FROM "+ my_user_)

In [0]:
# CLEAN UP RESOURCES
spark.sql("DROP CATALOG " + catalog_ + " CASCADE;")